# Scraping Burger Master

In [118]:
import os
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re 
from tqdm import tqdm
from geopy.geocoders import Nominatim

In [197]:
# Accedemos a la página oficial del Burger Master para Bogotá
url = "https://tuliorecomienda.com/2022/04/20/participantes-burger-master-2022-bogota/"
r = requests.get(url)
soup = BeautifulSoup(r.text, 'lxml')
elementos = soup.find("div", {"class": "elementor-section-wrap"})
secciones = elementos.find_all("section")
restaurantes = pd.DataFrame()
# La primera sección no nos importa, entonces nos saltamos el índice 0
# seccion = secciones[1]
for i in tqdm(range(1, len(secciones))):
    hamburguesas = secciones[i].next.next.next.next.find_all("div", {"data-element_type": "column"})
    # hamburguesa = hamburguesas[0]
    for hamburguesa in hamburguesas:
        nombre = hamburguesa.find("div", {"data-widget_type": "heading.default"})
        if nombre is None:
            continue
        else:
            nombre = nombre.text
        nombre = nombre.replace("\n", "").strip()
        direccion = hamburguesa.find("div", {"data-widget_type": "text-editor.default"}).text
        direccion = direccion.replace("\n", "").strip()
        descripcion = hamburguesa.find("div", {"role": "tabpanel"}).next
        if "Varios locales" in direccion:
            direcciones = descripcion
            direcciones = str(direcciones).split("<br/>")
            filtro1 = [len(re.findall('[0-9]+', d)) > 0 for d in direcciones]
            filtro2 = [not "<p>" in d for d in direcciones]
            filtro = [a & b for a, b in zip(filtro1, filtro2)]
            direcciones = [direcciones[j] for j in range(len(direcciones)) if filtro[j]]
            direcciones = [j.replace("</p>", "").strip() for j in direcciones]
            restaurante = pd.DataFrame([(nombre, d, descripcion) for d in direcciones], columns = ["Restaurante", "Dirección", "Descripción"])
        else:
            restaurante = pd.DataFrame([(nombre, direccion, descripcion)], columns = ["Restaurante", "Dirección", "Descripción"])
        restaurantes = pd.concat([restaurantes, restaurante]).reset_index(drop = True)
# Limpiamos algunos errores
restaurantes = restaurantes[~restaurantes.Dirección.apply(lambda x: (x[len(x) - 1] == ".") | (x[len(x) - 1] == "!"))].reset_index(drop = True)

100%|██████████| 28/28 [00:00<00:00, 304.28it/s]


In [198]:
# Vamos a convertir las direcciones en coordenadas
# Podemos usar la API de Google que es la más precisa, pero esto requiere configuración adicional. Por ende, iremos
# por la fácil que es GeoPy

locator = Nominatim(user_agent = "clase_MIAD")
restaurantes["Latitud"] = np.nan
restaurantes["Longitud"] = np.nan
restaurantes["Dirección"] = restaurantes["Dirección"].str.split(",", expand = True)[0]

# Olviden lo mencionado anteriormente. El resultado de usar GeoPy fue nefasto! Vamos a utilizar la API de Google

# for i in tqdm(range(restaurantes.shape[0])):
#     direccion = restaurantes["Dirección"][i] + ", Bogotá, Colombia"
#     coordenadas = locator.geocode(direccion, timeout = 60)
#     if coordenadas is not None:
#         restaurantes.loc[i, "Latitud"] = coordenadas.latitude
#         restaurantes.loc[i, "Longitud"] = coordenadas.longitude

## Existen algunos restaurantes que no pudimos localizar. No nos detendremos a arreglar su nomenclatura
# restaurantes = restaurantes[restaurantes["Latitud"].notnull()].reset_index(drop = True)

In [200]:
# Ingrese su API aquí. Puede sacarla de esta url: https://developers.google.com/maps
API_key = "ingrese_su_api_key_aquí" 
link_base = 'https://maps.googleapis.com/maps/api/geocode/json?address='

for i in tqdm(range(restaurantes.shape[0])):
    direccion = restaurantes["Dirección"][i] + ", Bogotá, Colombia"
    direccion = direccion.strip().replace("#", "").replace(" ", "+")
    url = link_base + direccion + "&key=" + API_key
    response = requests.get(url)
    resp_json_payload = response.json()
    restaurantes.loc[i, "Latitud"] = resp_json_payload['results'][0]['geometry']['location']["lat"]
    restaurantes.loc[i, "Longitud"] = resp_json_payload['results'][0]['geometry']['location']["lng"]

100%|██████████| 137/137 [00:39<00:00,  3.46it/s]


In [202]:
restaurantes.to_excel("burgar_master.xlsx", index = False)

In [106]:
# Podría incluirse un bono que consiste en reducir la mayor cantidad de NAs en la busqueda de coordenadas a partir 
# de la normalización del texto de las coordenadas.